In [2]:
import pandas as pd
import datetime
from os import listdir
import os, os.path
from os.path import isfile, join
import numpy as np
import csv
import re
import pandas as pd
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.log.exporter.xes import exporter as xes_exporter
from pm4py.objects.log.util import dataframe_utils
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
df_prueba = pd.read_csv("Log of analysis of time aspects.csv",index_col=0)

In [4]:
df_report_year = df_prueba.groupby(by=['Question', 'Year'])
for name, group in df_report_year:
    operations = group['concept:name'].to_list()
    

In [5]:
df_agrupado = pd.DataFrame(columns=['case:concept:name','Operations','Organization', 'Year', 'Question'])

In [6]:
df_analisis = df_prueba.groupby(by=['case:concept:name'])
# columns=['case:concept:name','Operations','Organization', 'Year', 'Question']
df_agrupado = pd.DataFrame(columns=['case:concept:name','Operations','Organization', 'Year', 'Question'])
for name, group in df_analisis:
    org = group.iloc[0]['Organization']
    year = group.iloc[0]['Year']
    question = group.iloc[0]['Question']
    lista = group['concept:name'].to_list()
        
    new_row={'case:concept:name':name, 'Operations':lista, 'Organization':org, 'Year':year, 'Question':question} 
    df_agrupado = df_agrupado.append(new_row, ignore_index=True)

In [7]:
df_agrupado

,case:concept:name,Operations,Organization,Year,Question
0,A1 P1 BPI 2017,"[Calculate cycle time, Calculate cycle time, F...",University of Liechtenstein\nVienna University...,2017,P1
1,A1 P1-P2 BPI 2020,"[Filter traces, Calculate cycle time, Calculat...","Technische Universität, Berlin",2020,P1-P2
2,A1 P2 BPI 2019,"[Group events by time, Calculate cycle time, R...",University of Melbourne,2019,P2
3,A1 P5 BPI 2015,"[Calculate cycle time, Identify organizational...",Meijer & Van der Ham Management Consultants,2015,P5
4,A1 P5 BPI 2020,"[Filter traces, Calculate cycle time, Find bot...","Technische Universität, Berlin",2020,P5
...,...,...,...,...,...
105,A8 P1 BPI 2017,"[Filter traces, Calculate waiting time, Repres...",Universidade Federal do Estado do Rio de Janeiro,2017,P1
106,A8 P5 BPI 2015,"[Calculate cycle time, Compare cycle time, Gro...",GRADIENT ECM,2015,P5
107,A9 P1 BPI 2017,"[Calculate processing time, Calculate waiting ...",POSTECH,2017,P1
108,A9 P2 BPI 2019,"[Filter traces, Calculate cycle time, Identify...",KPMG Netherlands,2019,P2


In [8]:
def jaccardIndex(list1,list2):
    set1=set(list1)
    set2=set(list2)
    intersection = set1.intersection(set2)
    tamIntersection=len(list(intersection))
    union = set1.union(set2)
    tamUnion=len(list(union))
#     print("similitud: "+str(float(tamIntersection)/tamUnion))
    return float(tamIntersection)/tamUnion

In [9]:
def overlapIndex(list1,list2):
    set1=set(list1)
    set2=set(list2)
    intersection = set1.intersection(set2)
    len1=len(list(set1))
    len2=len(list(set2))
#     print(float(len(list(intersection))))
#     print(min([len1,len2]))
    overlapIndex=float(len(list(intersection)))/min([len1,len2])
    return overlapIndex

In [10]:
def sorensenIndex(list1,list2):
    set1=set(list1)
    set2=set(list2)
    intersection = set1.intersection(set2)
    len1=len(list(set1))
    len2=len(list(set2))
    sorensenIndex=float(2*len(list(intersection)))/(len1+len2)
    return sorensenIndex

In [11]:
df_2020 = df_agrupado[df_agrupado['Year']==2020]
df2 = df_2020.groupby(by=['Year', 'Question'])
columns=['analisis','organizaciones','overlap','jaccard','sorensen']
df_metricas = pd.DataFrame(columns=columns)

for name, group in df2:
    for i in range(len(group)):
        if(group.iloc[i]['Organization'] != 'Sberbank'):
            org1 = 'No_Sberbank'
        else:
            org1 = 'Sberbank'
        
        for j in range(i+1,len(group)):
            grupo = group.iloc[i]['Operations']
            analisis = group.iloc[i]['case:concept:name'] + "-" + group.iloc[j]['case:concept:name']
                
            if(group.iloc[j]['Organization'] != 'Sberbank'):
                org2 = 'No_Sberbank'
            else:
                org2 = 'Sberbank'
            org = org1 + "-" + org2
            sorensen = sorensenIndex(grupo,group.iloc[j]['Operations'])
            overlap = overlapIndex(grupo,group.iloc[j]['Operations'])
            jaccard = jaccardIndex(grupo,group.iloc[j]['Operations'])
            new_row={'analisis':analisis,'organizaciones':org,'overlap':overlap,'jaccard':jaccard,'sorensen':sorensen} 
            df_metricas = df_metricas.append(new_row, ignore_index=True)
        
df_metricas = df_metricas.replace({'Sberbank-No_Sberbank':'No_Sberbank-Sberbank'})
df_metricas 

,analisis,organizaciones,overlap,jaccard,sorensen
0,A1 P1-P2 BPI 2020-A10 P1-P2 BPI 2020,No_Sberbank-No_Sberbank,0.750000,0.428571,0.600000
1,A1 P1-P2 BPI 2020-A11 P1-P2 BPI 2020,No_Sberbank-Sberbank,1.000000,0.571429,0.727273
2,A1 P1-P2 BPI 2020-A12 P1-P2 BPI 2020,No_Sberbank-Sberbank,0.500000,0.285714,0.444444
3,A1 P1-P2 BPI 2020-A13 P1-P2 BPI 2020,No_Sberbank-Sberbank,1.000000,0.444444,0.615385
4,A1 P1-P2 BPI 2020-A16 P1-P2 BPI 2020,No_Sberbank-No_Sberbank,0.750000,0.375000,0.545455
...,...,...,...,...,...
552,A30 P6 BPI 2020-A37 P6 BPI 2020,No_Sberbank-Sberbank,0.333333,0.142857,0.250000
553,A30 P6 BPI 2020-A7 P6 BPI 2020,No_Sberbank-No_Sberbank,0.666667,0.500000,0.666667
554,A35 P6 BPI 2020-A37 P6 BPI 2020,No_Sberbank-Sberbank,0.000000,0.000000,0.000000
555,A35 P6 BPI 2020-A7 P6 BPI 2020,No_Sberbank-No_Sberbank,0.500000,0.250000,0.400000


In [12]:
df_metricas.groupby(by=['organizaciones']).mean()

,overlap,jaccard,sorensen
organizaciones,,,
No_Sberbank-No_Sberbank,0.588952,0.313153,0.444988
No_Sberbank-Sberbank,0.510806,0.248390,0.359626
Sberbank-Sberbank,0.400998,0.175681,0.262647


In [13]:
from scipy import stats

group1 =  df_metricas[df_metricas['organizaciones'] == 'Sberbank-Sberbank']
group2 =  df_metricas[df_metricas['organizaciones'] == 'No_Sberbank-Sberbank']
group3 =  df_metricas[df_metricas['organizaciones'] == 'No_Sberbank-No_Sberbank']
# stats.kruskal(group1, group2, group3)

In [14]:
df_metricas.groupby(by=['organizaciones'])['overlap'].var(ddof=0)

organizaciones
No_Sberbank-No_Sberbank    0.078452
No_Sberbank-Sberbank       0.115245
Sberbank-Sberbank          0.134239
Name: overlap, dtype: float64

In [15]:
df_metricas.groupby(by=['organizaciones'])['overlap'].std(ddof=0)

organizaciones
No_Sberbank-No_Sberbank    0.280093
No_Sberbank-Sberbank       0.339478
Sberbank-Sberbank          0.366387
Name: overlap, dtype: float64

In [16]:
df_metricas.groupby(by=['organizaciones'])['jaccard'].var(ddof=0)

organizaciones
No_Sberbank-No_Sberbank    0.038601
No_Sberbank-Sberbank       0.039796
Sberbank-Sberbank          0.033071
Name: jaccard, dtype: float64

In [17]:
df_metricas.groupby(by=['organizaciones'])['jaccard'].std(ddof=0)

organizaciones
No_Sberbank-No_Sberbank    0.196470
No_Sberbank-Sberbank       0.199489
Sberbank-Sberbank          0.181854
Name: jaccard, dtype: float64

In [18]:
df_metricas.groupby(by=['organizaciones'])['sorensen'].var(ddof=0)

organizaciones
No_Sberbank-No_Sberbank    0.047393
No_Sberbank-Sberbank       0.059769
Sberbank-Sberbank          0.056963
Name: sorensen, dtype: float64

In [19]:
df_metricas.groupby(by=['organizaciones'])['sorensen'].std(ddof=0)

organizaciones
No_Sberbank-No_Sberbank    0.217698
No_Sberbank-Sberbank       0.244478
Sberbank-Sberbank          0.238669
Name: sorensen, dtype: float64